In [1]:
import os
import ctypes
from gtimer import GlobalTimer
from numpy.ctypeslib import ndpointer
import numpy as np

PLAN_LIB_PATH = "../cmake-build-debug/libmoveit_plan_compact.so"

from enum import Enum

class ObjectType(Enum):
    BOX = 1
    SPHERE = 2
    CYLINDER = 3

class ObjectAction(Enum):
    ADD = 0
    REMOVE = 1
    APPEND = 2
    MOVE = 3

clib = ctypes.cdll.LoadLibrary(PLAN_LIB_PATH)
MAX_STR_LEN = clib.get_max_str_len()
MAX_NAME_LEN = clib.get_max_name_len()
MAX_JOINT_NUM = clib.get_max_joint_num()
MAX_TRAJ_LEN = clib.get_max_traj_len()

class c_string(ctypes.Structure):
    _fields_ = [("buffer", ctypes.c_char*MAX_STR_LEN),
                ("len", ctypes.c_int)
                ]

class c_trajectory(ctypes.Structure):
    _fields_ = [("names_flt", ctypes.c_char*(MAX_JOINT_NUM*MAX_NAME_LEN)),
                ("joints", ctypes.c_double*(MAX_JOINT_NUM*MAX_TRAJ_LEN)),
                ("name_len", ctypes.c_int),
                ("joint_count", ctypes.c_int),
                ("joint_max", ctypes.c_int),
                ("traj_len", ctypes.c_int),
                ("success", ctypes.c_bool)
                ]

class c_plan_goal(ctypes.Structure):
    _fields_ = [("group_name", ctypes.c_char*MAX_NAME_LEN),
                ("tool_link", ctypes.c_char*MAX_NAME_LEN),
                ("goal_link", ctypes.c_char*MAX_NAME_LEN),
                ("goal_pose", ctypes.c_double*7),
                ("timeout", ctypes.c_double)
                ]

class c_object_msg(ctypes.Structure):
    _fields_ = [("name", ctypes.c_char*MAX_NAME_LEN),
                ("link_name", ctypes.c_char*MAX_NAME_LEN),
                ("pose", ctypes.c_double*7),
                ("dims", ctypes.c_double*3),
                ("type", ctypes.c_int),
                ("action", ctypes.c_int)
                ]

clib.hello_cstr.restype = c_string
clib.hello_char.restype = ndpointer(dtype=ctypes.c_char, shape=(MAX_STR_LEN,))
clib.plan_compact.restype = c_trajectory
clib.plan_compact.argtypes = [c_plan_goal]
clib.process_object.argtypes = [c_object_msg]

def convert_trajectory(traj):
    names = traj.names_flt.split()
    joints = []
    for i_traj in range(traj.traj_len):
        joints.append(traj.joints[i_traj*traj.joint_max:i_traj*traj.joint_max+traj.joint_count])
    return names, np.array(joints), traj.success


In [2]:
gtimer = GlobalTimer.instance()

In [3]:
urdf_path = "../test_assets/custom_robots.urdf"
srdf_path = "../test_assets/custom_robots.srdf"

urdf_str = c_string()
srdf_str = c_string()
with open(urdf_path, 'r') as f:
    while True:
        line = f.readline()
        urdf_str.buffer += line
        if not line: break
            
urdf_str.buffer
with open(srdf_path, 'r') as f:
    while True:
        line = f.readline()
        srdf_str.buffer += line
        if not line: break

In [4]:
gtimer.reset(1e3,'ms')
gtimer.tic("init_ros")
clib.init_planner(urdf_str, srdf_str)
gtimer.toc("init_ros")

99.3800163269043

In [5]:
gtimer.tic("add_object")
omsg = c_object_msg()
omsg.name = "box"
omsg.link_name = "base_link"
goal_pose = [-0.3,-0.2,0.0,0,0,0,1]
for igp in range(7): omsg.pose[igp] = goal_pose[igp]
dims = [0.1,0.1,0.1]
for igp in range(3): omsg.dims[igp] = dims[igp]
omsg.action = ObjectAction.ADD.value
omsg.type = ObjectType.BOX.value
clib.process_object(omsg)
gtimer.toc("add_object")

0.3249645233154297

In [6]:
for _ in range(10):
    gtimer.tic("plan_compact")
    goal = c_plan_goal()
    goal.group_name = "indy0"
    goal.tool_link = "indy0_tcp"
    goal_pose = [-0.3, -0.2, 0.4, 0, 0, 0, 1]
    for igp in range(7): goal.goal_pose[igp] = goal_pose[igp]
    goal.goal_link = "base_link"
    goal.timeout = 0.1
    c_traj = clib.plan_compact(goal)
    gtimer.toc("plan_compact")
    gtimer.tic("convert_trajectory")
    names, traj, succ = convert_trajectory(c_traj)
    gtimer.toc("convert_trajectory")
print(gtimer)
print(succ)

init_ros: 	99.0 ms/1 = 99.38 ms (99.38/99.38)
add_object: 	0.0 ms/1 = 0.325 ms (0.325/0.325)
plan_compact: 	423.0 ms/10 = 42.271 ms (28.104/80.765)
convert_trajectory: 	0.0 ms/10 = 0.05 ms (0.036/0.062)

True


In [7]:
gtimer.tic("clear_objects")
clib.clear_all_objects()
gtimer.toc("clear_objects")

0.1010894775390625

In [8]:
for _ in range(10):
    gtimer.tic("plan_compact")
    goal = c_plan_goal()
    goal.group_name = "indy0"
    goal.tool_link = "indy0_tcp"
    goal_pose = [-0.3, -0.2, 0.4, 0, 0, 0, 1]
    for igp in range(7): goal.goal_pose[igp] = goal_pose[igp]
    goal.goal_link = "base_link"
    goal.timeout = 0.1
    c_traj = clib.plan_compact(goal)
    gtimer.toc("plan_compact")
    gtimer.tic("convert_trajectory")
    names, traj, succ = convert_trajectory(c_traj)
    gtimer.toc("convert_trajectory")
print(gtimer)
print(succ)

init_ros: 	99.0 ms/1 = 99.38 ms (99.38/99.38)
add_object: 	0.0 ms/1 = 0.325 ms (0.325/0.325)
plan_compact: 	831.0 ms/20 = 41.562 ms (25.702/80.765)
convert_trajectory: 	1.0 ms/20 = 0.046 ms (0.034/0.062)
clear_objects: 	0.0 ms/1 = 0.101 ms (0.101/0.101)

True


In [9]:
gtimer.tic("add_object2")
omsg = c_object_msg()
omsg.name = "box"
omsg.link_name = "base_link"
goal_pose = [-0.3,-0.2,0.4,0,0,0,1]
for igp in range(7): omsg.pose[igp] = goal_pose[igp]
dims = [0.1,0.1,0.1]
for igp in range(3): omsg.dims[igp] = dims[igp]
omsg.action = ObjectAction.ADD.value
omsg.type = ObjectType.BOX.value
clib.process_object(omsg)
gtimer.toc("add_object2")

0.34809112548828125

In [10]:
for _ in range(10):
    gtimer.tic("plan_compact")
    goal = c_plan_goal()
    goal.group_name = "indy0"
    goal.tool_link = "indy0_tcp"
    goal_pose = [-0.3, -0.2, 0.4, 0, 0, 0, 1]
    for igp in range(7): goal.goal_pose[igp] = goal_pose[igp]
    goal.goal_link = "base_link"
    goal.timeout = 0.1
    c_traj = clib.plan_compact(goal)
    gtimer.toc("plan_compact")
    gtimer.tic("convert_trajectory")
    names, traj, succ = convert_trajectory(c_traj)
    gtimer.toc("convert_trajectory")
print(gtimer)
print(succ)

init_ros: 	99.0 ms/1 = 99.38 ms (99.38/99.38)
add_object: 	0.0 ms/1 = 0.325 ms (0.325/0.325)
plan_compact: 	1938.0 ms/30 = 64.587 ms (25.702/121.999)
convert_trajectory: 	160.0 ms/30 = 5.34 ms (0.034/24.189)
clear_objects: 	0.0 ms/1 = 0.101 ms (0.101/0.101)
add_object2: 	0.0 ms/1 = 0.348 ms (0.348/0.348)

False
